# SQL 的五十道練習

> 條件邏輯

郭耀仁 <yaojenkuo@datainpoint.com>，[數據交點](https://www.datainpoint.com)

In [1]:
import sqlite3
import unittest
import numpy as np
import pandas as pd
conn = sqlite3.connect('../databases/nba.db')
conn.execute("""ATTACH '../databases/covid19.db' AS covid19""")
conn.execute("""ATTACH '../databases/twElection2020.db' AS twElection2020""")
conn.execute("""ATTACH '../databases/imdb.db' AS imdb""")

## 從新冠肺炎的每日報告資料表 `daily_report` 中將美國與「非美國」的觀測值利用 `CASE` 敘述產生一個衍生欄位 `is_us` 區分出來，美國的觀測值給予布林真（`1`）、非美國的觀測值給予布林假（`0`），選擇 `Country_Region`、與 `is_us` 兩個欄位，並且以 `is_us` 遞減排序、`Country_Region` 遞增排序。

- 預期輸入：SQL 查詢語法。
- 預期輸出：(3979, 2) 的查詢結果。

```
          Country_Region  is_us
0                     US      1
1                     US      1
2                     US      1
3                     US      1
4                     US      1
...                  ...    ...
3974             Vietnam      0
3975  West Bank and Gaza      0
3976               Yemen      0
3977              Zambia      0
3978            Zimbabwe      0

[3979 rows x 2 columns]
```

In [2]:
case_is_us =\
"""
-- SQL 查詢語法起點
SELECT Country_Region,
       CASE WHEN Country_Region = 'US' THEN 1
            ELSE 0 END AS is_us
  FROM daily_report
 ORDER BY is_us DESC,
          Country_Region;
-- SQL 查詢語法終點
"""

## 從 IMDb 最高評價的 250 部電影資料表 `top_rated_movies` 中利用 `CASE` 敘述產生一個衍生欄位 `rating_category` 將評等超過 8.7（`>8.7`）的電影分類為 `'Awesome'`、將評等超過 8.4（`>8.4`）的電影分類為 `'Terrific'`，再將其餘的電影分類為 `'Great'`，選擇 `title`、`rating` 與 `rating_category`，以 `rating_category` 遞增排序、`rating` 遞減排序。

- 預期輸入：SQL 查詢語法。
- 預期輸出：(250, 3) 的查詢結果。

```
                            title  rating rating_category
0        The Shawshank Redemption     9.3         Awesome
1                   The Godfather     9.2         Awesome
2          The Godfather: Part II     9.0         Awesome
3                 The Dark Knight     9.0         Awesome
4                    12 Angry Men     9.0         Awesome
..                            ...     ...             ...
245                  The Prestige     8.5        Terrific
246  Once Upon a Time in the West     8.5        Terrific
247                    Casablanca     8.5        Terrific
248               Cinema Paradiso     8.5        Terrific
249                         Joker     8.5        Terrific

[250 rows x 3 columns]
```

In [3]:
case_rating_category =\
"""
-- SQL 查詢語法起點
SELECT title,
       rating,
       CASE WHEN rating > 8.7 THEN 'Awesome'
            WHEN rating > 8.4 THEN 'Terrific'
            ELSE 'Great' END AS rating_category
  FROM top_rated_movies
 ORDER BY rating_category,
          rating DESC;
-- SQL 查詢語法終點
"""

## 從台灣 2020 總統副總統的各投票所得票明細資料表 `presidential` 中利用 `CASE` 敘述產生一個衍生欄位 `county_type` 將觀測值分類為 `'六都'`與`'非六都'`，選擇 `county` 與 `county_type` 欄位。

- 預期輸入：SQL 查詢語法。
- 預期輸出：(51678, 2) 的查詢結果。

註：六都為臺北市、新北市、桃園市、臺中市、臺南市與高雄市。

```
      county county_type
0        宜蘭縣         非六都
1        宜蘭縣         非六都
2        宜蘭縣         非六都
3        宜蘭縣         非六都
4        宜蘭縣         非六都
...      ...         ...
51673    臺中市          六都
51674    臺中市          六都
51675    臺中市          六都
51676    臺中市          六都
51677    臺中市          六都

[51678 rows x 2 columns]
```

In [4]:
case_county_type =\
"""
-- SQL 查詢語法起點
SELECT county,
       CASE WHEN county IN ('臺北市', '新北市', '桃園市', '臺中市', '臺南市', '高雄市') THEN '六都'
            ELSE '非六都' END AS county_type
  FROM presidential;
-- SQL 查詢語法終點
"""

## 執行測試！

Kernel -> Restart & Run All.

In [5]:
class TestCase(unittest.TestCase):
    def test_case_is_us(self):
        is_us = pd.read_sql(case_is_us, conn)
        self.assertEqual(is_us.shape, (3979, 2))
        self.assertEqual(is_us['is_us'].sum(), 3275)
    def test_case_rating_category(self):
        rating_category = pd.read_sql(case_rating_category, conn)
        self.assertEqual(rating_category.shape, (250, 3))
        np.testing.assert_equal(rating_category['rating_category'].values[:5],
                               np.array(['Awesome', 'Awesome', 'Awesome', 'Awesome', 'Awesome']))
        np.testing.assert_equal(rating_category['rating_category'].values[-5:],
                               np.array(['Terrific', 'Terrific', 'Terrific', 'Terrific', 'Terrific']))
        
    def test_case_county_type(self):
        county_type = pd.read_sql(case_county_type, conn)
        self.assertEqual(county_type.shape, (51678, 2))
        np.testing.assert_equal(county_type['county_type'].unique(),
                               np.array(['非六都', '六都']))
        np.testing.assert_equal(county_type['county_type'].values[:5],
                               np.array(['非六都', '非六都', '非六都', '非六都', '非六都']))
        np.testing.assert_equal(county_type['county_type'].values[-5:],
                               np.array(['六都', '六都', '六都', '六都', '六都']))
        
suite = unittest.TestLoader().loadTestsFromTestCase(TestCase)
runner = unittest.TextTestRunner(verbosity=2)
test_results = runner.run(suite)
number_of_failures = len(test_results.failures)
number_of_errors = len(test_results.errors)
number_of_test_runs = test_results.testsRun
number_of_successes = number_of_test_runs - (number_of_failures + number_of_errors)

test_case_county_type (__main__.TestCase) ... ok
test_case_is_us (__main__.TestCase) ... ok
test_case_rating_category (__main__.TestCase) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.264s

OK


In [6]:
print("您在 {} 道 SQL 練習中答對了 {} 題。".format(number_of_test_runs, number_of_successes))

您在 3 道 SQL 練習中答對了 3 題。
